# FinGuard IntelliAgent - Milestone 6: End-to-End Orchestration

**Author:** Alfred Munga  
**Date:** January 2025  
**License:** MIT

---

## Overview

This notebook demonstrates **Milestone 6** of the FinGuard IntelliAgent ADK capstone project:

### 🎯 Key Concepts Implemented

1. **The "Think, Act, Observe" Loop** (Intro to Agents p.11)
   - Model reasons about the query
   - Selects and executes appropriate tools
   - Observes results and decides next steps

2. **Context Lifecycle** (Context Engineering p.9)
   - Fetch Context → Prepare Prompt → Invoke Model → Update Session
   - Memory management across conversations
   - Session state persistence

3. **Observability (Traces)** (Prototype to Production p.30)
   - Structured logging of reasoning and actions
   - Complete execution trajectory
   - Debugging and performance analysis

---

## Architecture

```
User Query
    ↓
┌───────────────────────────────────┐
│  STEP 1: Fetch Context           │
│  - User Profile                   │
│  - Conversation History           │
│  - Memory Bank                    │
└───────────┬───────────────────────┘
            ↓
┌───────────────────────────────────┐
│  STEP 2: Prepare Prompt           │
│  - System Instruction (Persona)   │
│  - Context Integration            │
│  - Tool Descriptions              │
└───────────┬───────────────────────┘
            ↓
┌───────────────────────────────────┐
│  STEP 3: Think-Act-Observe Loop   │
│                                    │
│  THINK: Model reasons              │
│    ↓                               │
│  ACT: Execute tool                 │
│    ↓                               │
│  OBSERVE: Process result           │
│    ↓                               │
│  (Repeat if needed)                │
└───────────┬───────────────────────┘
            ↓
┌───────────────────────────────────┐
│  STEP 4: Update Context            │
│  - Save to Session                 │
│  - Log Trajectory                  │
└───────────┬───────────────────────┘
            ↓
        Response
```

---

Let's begin!

## 🚀 Setup

Import the orchestrator and initialize components.

In [ ]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Load environment variables
load_dotenv()

# Import orchestrator and logger
from agent.orchestrator import FinGuardIntelliAgent
from backend.utils.logger import AgentLogger
from agent.memory import MemoryBank, UserProfile

# Get API key
api_key = os.getenv('GEMINI_API_KEY')
if not api_key:
    raise ValueError("GEMINI_API_KEY not found in .env file")

print("✅ Setup complete!")
print(f"📂 Project root: {project_root}")

## 🤖 Initialize the Agent

Create the FinGuard IntelliAgent with all tools and memory.

In [ ]:
# Initialize memory with user profile
user_profile = UserProfile(
    name="Jane Doe",
    business_type="Retail",
    location="Nairobi, Kenya"
)

memory = MemoryBank(
    user_profile=user_profile,
    budgets={
        "transport": 5000,
        "utilities": 3000,
        "supplies": 10000
    }
)

# Initialize agent
agent = FinGuardIntelliAgent(
    api_key=api_key,
    memory=memory,
    max_iterations=5  # Prevent infinite loops
)

print("=" * 70)
print("🤖 FinGuard IntelliAgent Initialized")
print("=" * 70)
print(f"User: {user_profile.name}")
print(f"Business: {user_profile.business_type}")
print(f"Available Tools: 4 (SMS Parser, Insights, Invoice Ops, Payment Request)")
print(f"Max Iterations: 5")
print("=" * 70)

## 📋 Scenario 1: Information Retrieval

**Query:** "Who owes me money?"

**Expected Flow:**
1. **THINK**: Agent reasons that user wants unpaid invoices
2. **ACT**: Calls `get_unpaid_invoices` tool
3. **OBSERVE**: Receives list of unpaid invoices
4. **THINK**: Formats response for user
5. **FINAL**: Returns structured answer

**⚠️ Note:** This may take 10-30 seconds due to API calls.

In [ ]:
print("=" * 70)
print("📋 Scenario 1: Information Retrieval")
print("=" * 70)

# Create logger for this run
logger = AgentLogger()

# Run agent
result = agent.run(
    user_query="Who owes me money? Show me my unpaid invoices.",
    user_id="jane_doe",
    trace_logger=logger
)

# Display results
if result['success']:
    print("\n✅ SUCCESS!")
    print("\n📝 Agent Response:")
    print("-" * 70)
    print(result['response'])
    print("-" * 70)
    
    # Show summary
    print("\n📊 Execution Summary:")
    summary = result['summary']
    print(f"  Trace ID: {summary['trace_id']}")
    print(f"  Total Steps: {summary['total_steps']}")
    print(f"  Tools Used: {summary['act_count']} ({', '.join(set(summary['tools_used']))})")
    print(f"  Duration: {summary.get('duration', 'N/A')}s")
else:
    print("\n❌ FAILED")
    print(f"Error: {result.get('error')}")

print("\n" + "=" * 70)

## 🔍 View Execution Trajectory

This demonstrates the **Observability** concept - we can see exactly how the agent reasoned and acted.

In [ ]:
# Print the complete trajectory
logger.print_trajectory()

## 💳 Scenario 2: Multi-Step Action Flow

**Complex Query:** "I received an M-Pesa message: 'QQG5KNERRX Confirmed. Ksh5,000.00 received from JOHN DOE 254712345678 on 18/11/24'. Can you check if this clears any of John's invoices?"

**Expected Flow:**
1. **THINK**: Agent identifies two tasks - parse SMS and check invoices
2. **ACT**: Calls `parse_sms` tool
3. **OBSERVE**: Gets transaction details (sender: JOHN DOE, amount: KES 5,000)
4. **THINK**: Now check for John's unpaid invoices
5. **ACT**: Calls `get_unpaid_invoices` tool
6. **OBSERVE**: Gets list of unpaid invoices
7. **THINK**: Matches payment to invoice (if applicable)
8. **FINAL**: Provides comprehensive answer

This demonstrates the multi-step reasoning capability!

In [ ]:
print("=" * 70)
print("💳 Scenario 2: Multi-Step Action Flow")
print("=" * 70)

# Create new logger
logger2 = AgentLogger()

# Complex query requiring multiple tools
complex_query = """
I just received this M-Pesa message:

'QQG5KNERRX Confirmed. Ksh5,000.00 received from JOHN DOE 254712345678 on 18/11/24 at 2:30 PM'

Can you:
1. Parse this message
2. Check if John DOE has any unpaid invoices
3. Let me know if this payment covers any of his invoices
"""

# Run agent
result2 = agent.run(
    user_query=complex_query,
    user_id="jane_doe",
    trace_logger=logger2
)

# Display results
if result2['success']:
    print("\n✅ SUCCESS!")
    print("\n📝 Agent Response:")
    print("-" * 70)
    print(result2['response'])
    print("-" * 70)
    
    # Show summary
    print("\n📊 Execution Summary:")
    summary2 = result2['summary']
    print(f"  Trace ID: {summary2['trace_id']}")
    print(f"  Total Steps: {summary2['total_steps']}")
    print(f"  Tools Used: {summary2['act_count']}")
    print(f"  Tool Sequence: {' → '.join(summary2['tools_used'])}")
    print(f"  Duration: {summary2.get('duration', 'N/A')}s")
else:
    print("\n❌ FAILED")
    print(f"Error: {result2.get('error')}")

print("\n" + "=" * 70)

## 📊 View Multi-Step Trajectory

This trajectory should show multiple tool calls in sequence.

In [ ]:
# Print trajectory for the multi-step query
logger2.print_trajectory()

## 🔒 Scenario 3: Action with Idempotency

**Query:** "Send a payment request to the most overdue customer."

**Expected Flow:**
1. **THINK**: Need to find most overdue invoice first
2. **ACT**: Calls `get_unpaid_invoices`
3. **OBSERVE**: Gets list sorted by days overdue
4. **THINK**: Confirms user wants to send payment request
5. **ACT**: Calls `send_payment_request` with invoice ID
6. **OBSERVE**: Gets payment request result (or idempotency rejection)
7. **FINAL**: Informs user of outcome

**⚠️ Important:** If you run this cell twice, the second attempt should be rejected due to idempotency protection!

In [ ]:
print("=" * 70)
print("🔒 Scenario 3: Action with Idempotency")
print("=" * 70)

# Create new logger
logger3 = AgentLogger()

# Action query
action_query = """
Send a payment request to the customer with the most overdue invoice.
Yes, I confirm I want to do this.
"""

# Run agent
result3 = agent.run(
    user_query=action_query,
    user_id="jane_doe",
    trace_logger=logger3
)

# Display results
if result3['success']:
    print("\n✅ SUCCESS!")
    print("\n📝 Agent Response:")
    print("-" * 70)
    print(result3['response'])
    print("-" * 70)
    
    # Show summary
    print("\n📊 Execution Summary:")
    summary3 = result3['summary']
    print(f"  Trace ID: {summary3['trace_id']}")
    print(f"  Tools Used: {summary3['act_count']}")
    print(f"  Tool Sequence: {' → '.join(summary3['tools_used'])}")
    
    # Check if payment was sent
    if 'send_payment_request' in summary3['tools_used']:
        print("\n💳 Payment request was initiated!")
        print("⚠️  If you run this cell again, it should be REJECTED (idempotency)")
else:
    print("\n❌ FAILED")
    print(f"Error: {result3.get('error')}")

print("\n" + "=" * 70)

## 📚 Summary: ADK Principles in Action

### 1. ✅ Think-Act-Observe Loop (Intro to Agents p.11)

**What we demonstrated:**
- Agent reasons about queries (THINK)
- Selects and executes appropriate tools (ACT)
- Processes results and decides next steps (OBSERVE)
- Iterates until task is complete

**Evidence:**
- Single-tool scenarios: 1 iteration
- Multi-step scenarios: 2-3 iterations
- Complete trajectories logged

---

### 2. ✅ Context Lifecycle (Context Engineering p.9)

**Lifecycle Steps:**
1. **Fetch Context**: User profile, conversation history
2. **Prepare Prompt**: System instruction + context
3. **Invoke Model**: Send to Gemini with tools
4. **Update Session**: Save conversation history

**Personalization:**
- User profile: "Jane Doe, Retail, Nairobi"
- Budgets: Transport (KES 5,000), Utilities (KES 3,000)
- Session history: Persists across queries

---

### 3. ✅ Observability (Prototype to Production p.30)

**What we logged:**
- Every reasoning step (THINK)
- Every tool call with inputs (ACT)
- Every tool result (OBSERVE)
- Complete execution trajectory

**Benefits:**
- **Debugging**: See exactly where failures occur
- **Performance**: Measure tool execution times
- **Auditing**: Track all agent actions
- **Analysis**: Understand decision patterns

**Log Files:**
- Location: `logs/agent_trace_YYYYMMDD.jsonl`
- Format: JSON Lines (one log entry per line)
- Retention: Persistent for analysis

---

## 🎯 Key Achievements

1. ✅ **Multi-Tool Coordination**: Agent successfully chains tools
2. ✅ **Context Management**: Memory persists across conversations
3. ✅ **Error Handling**: Graceful failure recovery
4. ✅ **Idempotency**: Actions protected against duplicates
5. ✅ **Observability**: Complete execution traces

---

## 🚀 What's Next?

### Production Deployment
1. Scale to handle concurrent users
2. Add authentication and authorization
3. Implement rate limiting
4. Set up monitoring and alerts

### Advanced Features
1. **Multi-Agent Systems**: Coordinate specialized agents
2. **Advanced Planning**: Long-horizon task decomposition
3. **Learning**: Adapt based on user feedback
4. **Proactive Assistance**: Suggest actions before user asks

### Tool Expansion
1. Add more financial tools (forecasting, reporting)
2. Integrate with real banking APIs
3. Support for multiple currencies
4. Advanced analytics and insights

---

## 📖 References

- **Intro to Agents**: Think-Act-Observe loop (p.11), Persona design (p.23)
- **Context Engineering**: Context lifecycle (p.9), Memory management (p.58)
- **Prototype to Production**: Observability patterns (p.30), Error handling
- **Agent Tools**: Tool design principles (p.18), Idempotency (p.19)

---

**End of Milestone 6 Demo**